In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import glob
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import torch
# add code directory to path
import sys
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from torch_models import NegativeBinomialRegressionModel
from metrics import top_k_onehot_indicator
from functools import partial
from torch_models import NegativeBinomialRegressionModel
from metrics import top_k_onehot_indicator
from torch_perturb.perturbations import perturbed
from torch_models import torch_bpr_uncurried, deterministic_bpr


2024-10-29 13:46:01.142738: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 13:46:01.195150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 13:46:01.195194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 13:46:01.196410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 13:46:01.205126: I tensorflow/core/platform/cpu_feature_guar

In [5]:
data_dir = '/cluster/home/kheuto01/code/prob_diff_topk'
base_dir = '/cluster/tufts/hugheslab/kheuto01/neg_binom_experiments_K100_8000'

In [8]:
def convert_df_to_3d_array(df):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(df.index, pd.MultiIndex) or set(df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Get unique geoids and timesteps, sorted
    geoids = sorted(df.index.get_level_values('geoid').unique())
    timesteps = sorted(df.index.get_level_values('timestep').unique())

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Initialize the 3D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    num_features = len(df.columns)
    X = np.zeros((num_timesteps, num_locations, num_features))

    # Fill the 3D array
    for (geoid, timestep), row in df.iterrows():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        X[t_idx, g_idx, :] = row.values

    return X, geoids, timesteps

def convert_y_df_to_2d_array(y_df, geoids, timesteps):
    # Ensure the DataFrame has a MultiIndex with 'geoid' and 'timestep'
    if not isinstance(y_df.index, pd.MultiIndex) or set(y_df.index.names) != {'geoid', 'timestep'}:
        raise ValueError("DataFrame must have a MultiIndex with levels 'geoid' and 'timestep'")

    # Initialize the 2D array
    num_timesteps = len(timesteps)
    num_locations = len(geoids)
    y = np.zeros((num_timesteps, num_locations))

    # Create a mapping of geoids to indices
    geoid_to_idx = {geoid: idx for idx, geoid in enumerate(geoids)}

    # Fill the 2D array
    for (geoid, timestep), value in y_df.iloc[:, 0].items():
        t_idx = timesteps.index(timestep)
        g_idx = geoid_to_idx[geoid]
        y[t_idx, g_idx] = value

    return y

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# Load training data
train_X_df = pd.read_csv(os.path.join(data_dir, 'mass_train_x.csv'), index_col=[0,1])
train_Y_df = pd.read_csv(os.path.join(data_dir, 'mass_train_y.csv'), index_col=[0,1])

# Load validation data
val_X_df = pd.read_csv(os.path.join(data_dir, 'mass_valid_x.csv'), index_col=[0,1])
val_Y_df = pd.read_csv(os.path.join(data_dir, 'mass_valid_y.csv'), index_col=[0,1])

# Process training data
train_X, geoids, timesteps = convert_df_to_3d_array(train_X_df.drop(columns='timestep.1'))
train_time_arr = np.array([timesteps] * len(geoids)).T
train_y = convert_y_df_to_2d_array(train_Y_df, geoids, timesteps)

# Process validation data
val_X, _, val_timesteps = convert_df_to_3d_array(val_X_df.drop(columns='timestep.1'))
val_time_arr = np.array([val_timesteps] * len(geoids)).T
val_y = convert_y_df_to_2d_array(val_Y_df, geoids, val_timesteps)

# Convert to tensors and move to device
X_train = torch.tensor(train_X, dtype=torch.float32).to(device)
y_train = torch.tensor(train_y, dtype=torch.float32).to(device)
time_train = torch.tensor(train_time_arr, dtype=torch.float32).to(device)

X_val = torch.tensor(val_X, dtype=torch.float32).to(device)
y_val = torch.tensor(val_y, dtype=torch.float32).to(device)
time_val = torch.tensor(val_time_arr, dtype=torch.float32).to(device)

In [12]:
T, S, F = X_train.shape

In [14]:
model = NegativeBinomialRegressionModel(
        num_locations=len(geoids),
        num_fixed_effects=train_X.shape[2]
    ).to(device)

In [16]:
param_vec = model.params_to_single_tensor()

In [18]:
param_vec.shape

torch.Size([3262])

In [19]:
S*2+5+F

3262